# Vlasiator - A Global Hybrid-Vlasov Simulation Model 
Vlasiator [@palmroth2018] is an open-source simulation software used to model the behavior of plasma in the Earth's magnetosphere, a region of space where the solar wind interacts with the Earth’s magnetic field. Vlasiator models collisionless space plasma dynamics by solving the 6-dimensional Vlasov equation, using a hybrid-Vlasov approach. It uses a 3D Cartesian grid in real space, with each cell storing another 3D Cartesian grid in velocity space. The velocity mesh contained in each spatial cell in the simulation domain has been represented so far by a sparse grid approach, fundamentally based on an associative container such as a key-value hashtable. Storing a 3D VDF at every spatial cells increases the memory requirements exponentially both during runtime and for storing purposes. Our proposal revolves around developing innovative solutions to compressing the VDFs during runtmime.

![title](images/egi.png)


# VDF Compression

## Initialization
Let's read in a vdf from a sample file and see what that looks like.

In [ ]:
import sys,os
# sys.path.append('/home/mjalho/analysator')
import tools as project_tools
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
import matplotlib.colors as colors
# plt.rcParams['figure.figsize'] = [7, 7]
import ctypes
import pyzfp,zlib
import mlp_compress
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import pytools

In [ ]:
file="../assets/bulk1.0001280.vlsv";cid=356780649;
#Read the VDF into a 3D uniform mesh and plot it
vdf=project_tools.extract_vdf(file,cid,25)
# np.save("sample_vdf.bin",np.array(vdf,dtype=np.double));
np.array(vdf,dtype=np.double).tofile("sample_vdf.bin")
nx,ny,nz=np.shape(vdf)
print(f"VDF shape = {np.shape(vdf)}")
fig = plt.figure(figsize=[7,4], dpi=300);
ax1 = plt.subplot(221)
ax2 = plt.subplot(222)
ax3 = plt.subplot(223)
ax4 = plt.subplot(224, projection='3d')
# , ax2, ax3) = plt.subplots(1, 3)


cmap = 'viridis'
cmap = 'hawaii_r'
norm = colors.LogNorm(vmin=1e-15,vmax=3e-13)
im1 = ax1.pcolormesh(vdf[:,:,nz//2],norm=norm,rasterized=False,cmap=cmap)
# im1 = ax1.imshow(vdf[:,:,nz//2],norm=norm,cmap=cmap, origin='lower')
im2 = ax2.pcolormesh(vdf[nx//2,:,:].T,norm=norm,rasterized=False,cmap=cmap)
im3 = ax3.pcolormesh(vdf[:,ny//2,:],norm=norm,rasterized=False,cmap=cmap)
ax1.axis('equal')
ax1.set_xlabel('vx')
ax1.set_ylabel('vy')
ax2.axis('equal')
ax2.set_xlabel('vz')
ax2.set_ylabel('vy')
ax3.axis('equal')
ax3.set_xlabel('vx')
ax3.set_ylabel('vz')

level = 1e-14
for level in [3e-15,1e-14,3e-14,1e-13,2e-13]:
    verts, faces, normals, values = measure.marching_cubes(vdf, level)
    mesh = Poly3DCollection(verts[faces], shade=True, facecolors=mpl.colormaps[cmap](norm(level)), alpha = norm(level)**2)
    ax4.add_collection3d(mesh)

ax4.set_xlim(nx/4,3*nx/4)
ax4.set_ylim(ny/4,3*ny/4)
ax4.set_zlim(nz/4,3*nz/4)
ax4.set_xlabel('vx')
ax4.set_ylabel('vy')
ax4.set_zlabel('vz')

# cax = fig.add_axes([0.7,0.1,0.2,0.05])
# fig.colorbar(im1, cax=cax, location='bottom')
fig.suptitle("Original VDF")
plt.show()

project_tools.plot_vdf_discrete_laplacians(vdf)

The vdf shown above is sampled on a uniform 3D velocity mesh and contains 64bit floating point numbers that represent the phase space density. We can calculate the total size of this VDF is bytes using ```sys.getsizeof(vdf)```.

In [ ]:
vdf_mem=sys.getsizeof(vdf)
num_stored_elements=len(vdf[vdf>1e-15])
print(f"VDF takes {vdf_mem} B.")

Now in Vlasiator we have countlesss VDFs since there is one per spatial cell. It would be great if we could compress them efficiently.

## Compression algorithms

### Zlib: lossless floating point compression
We can try to do so by using zlib which is a form of lossless compression. The reconstruction is accurate, by definition, but the compression ratio is small.

In [ ]:
compressed_vdf = zlib.compress(vdf)
compressed_vdf_mem=len(compressed_vdf)
compression_ratio=vdf_mem/compressed_vdf_mem
print(f"Achieved compression ratio using zlib= {round(compression_ratio,2)}.")
decompressed_vdf = zlib.decompress(compressed_vdf)
recon = np.frombuffer(decompressed_vdf, dtype=vdf.dtype).reshape(vdf.shape)
project_tools.plot_vdfs(vdf,recon)
project_tools.print_comparison_stats(vdf,recon)

### zfp: lossy floating point compression
We can use a lossy compression method like zfp[@zfp] to get even higher compression ratios.

In [ ]:
"""
Compresses a VDF  using ZFP (Zstandard Compressed FP)
Input:VDF - numpy array 
Output: recon (Reconstructed VDF) - numpy array 
"""
tolerance = 1e-13
compressed_vdf = pyzfp.compress(vdf, tolerance=tolerance)
compressed_vdf_mem=len(compressed_vdf)
compression_ratio=vdf_mem/compressed_vdf_mem
print(f"Achieved compression ratio using zfp= {round(compression_ratio,2)}.")
recon = pyzfp.decompress(compressed_vdf,vdf.shape,vdf.dtype,tolerance)
project_tools.plot_vdfs(vdf,recon)
project_tools.print_comparison_stats(vdf,recon)

### Multilevel Perceptron (MLP)

This is based on [@park2019].

In [ ]:
"""
Compresses a VDF using an MLP (Multilayer Perceptron).
 Input: "sample_vdf.bin" - Binary file containing the VDF data
        order - Order of the fourier features
        epochs - Number of training epochs for the MLP model
        n_layers - Number of layers in the MLP model
        n_neurons - Number of neurons in each layer of the MLP model
 Output: recon (Reconstructed VDF) - NumPy array representing the reconstructed volume data
"""
order=0
epochs=10
n_layers=4
n_neurons=25
recon=mlp_compress.compress_mlp("sample_vdf.bin",order,epochs,n_layers,n_neurons)
recon=np.array(recon,dtype=np.double)
recon= np.reshape(recon,np.shape(vdf),order='C')
project_tools.plot_vdfs(vdf,recon)
project_tools.print_comparison_stats(vdf,recon)

### MLP with Fourier features 

We will compress the VDF using an MLP with Fourier Features, which significantly improves MLP performance [@2020fourier].

In [ ]:
"""
Compresses a VDF using an MLP (Multilayer Perceptron).
 Input: "sample_vdf.bin" - Binary file containing the VDF data
        order - Order of the fourier features
        epochs - Number of training epochs for the MLP model
        n_layers - Number of layers in the MLP model
        n_neurons - Number of neurons in each layer of the MLP model
 Output: recon (Reconstructed VDF) - NumPy array representing the reconstructed volume data
"""
order=16
epochs=12
n_layers=4
n_neurons=25
recon=mlp_compress.compress_mlp("sample_vdf.bin",order,epochs,n_layers,n_neurons)
recon=np.array(recon,dtype=np.double)
recon= np.reshape(recon,np.shape(vdf),order='C')
project_tools.plot_vdfs(vdf,recon)
project_tools.print_comparison_stats(vdf,recon)

### Spherical Harmonic Decomposition

Spherical harmonics have been suggested as a usable approximation for VDFs in our subject domain [@vinas_gurgiolo_2009]. Here we investigate if the method can be employed as a compression method. This prototype could be improved by using e.g., the Misner [@misner_2004] method.

In [ ]:
"""
Compresses a VDF using a spherical harmonic decompostion
Input: "sample_vdf.bin" - Binary file containing the VDF data
       degree - Degree of the spherical harmonic decomposition (l)
Output: recon (Reconstructed VDF) - NumPy array representing the reconstructed volume data
"""

degree=10
recon=mlp_compress.compress_sph("sample_vdf.bin",degree)
recon=np.array(recon,dtype=np.double)
recon= np.reshape(recon,np.shape(vdf),order='C')
project_tools.plot_vdfs(vdf,recon)
project_tools.print_comparison_stats(vdf,recon)


### Convolutional Neural Network (CNN)

In [ ]:
"""
Function: train_and_reconstruct

Description:
This function takes an input array and trains a Convolutional Neural Network (CNN) model to reconstruct the input array.
It uses Mean Squared Error (MSE) loss and the Adam optimizer for training.

Inputs:
- input_array (numpy array): The input array to be reconstructed.
- num_epochs (int, optional): The number of training epochs.
- learning_rate (float, optional): The learning rate for the Adam optimize.
Outputs:
    Reconstructed vdf array 
    Size of model in bytes
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv3d(64, 1, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.conv4(x)
        return x

def train_and_reconstruct(input_array, num_epochs=30, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_tensor = torch.tensor(input_array, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # Add batch and channel dimensions, move to device
    model = CNN().to(device)  # Move model to device
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output_tensor = model(input_tensor)
        loss = criterion(output_tensor, input_tensor)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 100== 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    with torch.no_grad():
        output_tensor = model(input_tensor)
    reconstructed_array = output_tensor.squeeze(0).squeeze(0).cpu().numpy()
    
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size = (param_size + buffer_size)   
    return reconstructed_array, size


vdf_temp=vdf.copy()
vdf_temp[vdf_temp<1e-16]=1e-16
vdf_temp = np.log10(vdf_temp)
input_array=vdf_temp
recon,total_size= train_and_reconstruct(input_array,100)
recon = 10 ** recon
recon[recon <= 1e-16] = 0
vdf_size=nx*ny*nz*8
print(f"Compresion achieved using a CNN = {round(vdf_size/total_size,2)}")
project_tools.plot_vdfs(vdf,recon)
project_tools.print_comparison_stats(vdf,recon)

#### CNN with minibatches

Here we still use a CNN but this time we use minibatch training and batch normalization layers to try and improve performance.

In [ ]:
"""
Function: train_and_reconstruct

Description:
This function takes an input array and trains a Convolutional Neural Network (CNN) model to reconstruct the input array.
It uses Mean Squared Error (MSE) loss and the Adam optimizer for training.

Inputs:
- input_array (numpy array): The input array to be reconstructed.
- num_epochs (int, optional): The number of training epochs.
- learning_rate (float, optional): The learning rate for the Adam optimizer
Outputs:
    Reconstructed vdf array 
    Size of model in bytes
"""
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(16)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(32)
        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(64)
        self.conv4 = nn.Conv3d(64, 1, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        return x

def train_and_reconstruct(input_array, num_epochs=30, learning_rate=0.001, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_tensor = torch.tensor(input_array, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # Move input tensor to device
    model = CNN().to(device) 
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        for i in range(0, input_tensor.size(0), batch_size):
            optimizer.zero_grad()
            batch_input = input_tensor[i:i+batch_size]
            output_tensor = model(batch_input)
            loss = criterion(output_tensor, batch_input)
            loss.backward()
            optimizer.step()
    
    with torch.no_grad():
        output_tensor = model(input_tensor)
    reconstructed_array = output_tensor.squeeze(0).squeeze(0).cpu().numpy()
    
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size = (param_size + buffer_size)   
    return reconstructed_array, size

vdf_temp = vdf.copy()
vdf_temp[vdf_temp < 1e-16] = 1e-16
vdf_temp = np.log10(vdf_temp)
input_array = vdf_temp
recon, total_size = train_and_reconstruct(input_array, 100)

recon = 10 ** recon
recon[recon <= 1e-16] = 0
vdf_size = nx * ny * nz * 8
print(f"Compression achieved using a CNN = {round(vdf_size / total_size, 2)}")

project_tools.plot_vdfs(vdf, recon)
project_tools.print_comparison_stats(vdf, recon)


### Hermite Decomposition 

In [ ]:
"""
Loads the original 3D VDF and fits it to a Maxwellian distribution.
  Input: vdf - numpy array representing the original 3D VDF
  Output: vdf_herm_3d Reconstructed VDF using Hermite Decomposition
"""
### load original 3d vdf and fit Maxwellian
vdf_3d=vdf.copy()
print('loading done')
vdf_size=nx*ny*nz*8

#### Fit Maxwellian
v_min,v_max,n_bins=0,nx,nx ### define limits and size of velocity axes

amp,ux,uy,uz,vthx,vthy,vthz=1e-14,nx,nx,nx,10,10,10 ### initial guess for scipy curve fit
guess=amp,ux,uy,uz,vthx,vthy,vthz ### initial guess for scipy curve fit

max_fit_3d,params=project_tools.max_fit(vdf_3d,v_min,v_max,n_bins,guess) ### fitting
print('Maxwell fit done')


#### forward transform ####
mm=15 ### PUT THE NUMBER OF HARMONICS
norm_amp,u,vth=params[0],params[1:4],params[4:7] ### get the maxwellin fit parameters of thermal and bulk velocity

vdf_3d_norm=vdf_3d/norm_amp ### normalize data
vdf_3d_flat= vdf_3d_norm.flatten() ### flatten data

v_xyz=project_tools.get_flat_mesh(v_min,v_max,n_bins) ### flattening the mesh nodes coordinates
herm_array=np.array(project_tools.herm_mpl_arr(m_pol=mm,v_ax=v_xyz,u=params[1:4],vth=params[4:7])) ### create array of hermite polynomials

hermite_matrix=project_tools.coefficient_matrix(vdf_3d_flat,mm,herm_array,v_xyz) ### calculate the coefficients of the Hermite transform
print('Forward transform done')
total_size =5*8+8*np.prod(np.shape(hermite_matrix))

#### inverse transform ####
inv_herm_flat=project_tools.inv_herm_trans(hermite_matrix, herm_array, v_xyz) ### inverse Hermite transform
vdf_herm_3d = (np.reshape(inv_herm_flat,(n_bins,n_bins,n_bins)))*norm_amp ### reshaping back to 3d array and renormalization
print('Inverse transform done')
print(f"Compresion achieved using Hermite = {round(vdf_size/total_size,2)}")
project_tools.plot_vdfs(vdf,vdf_herm_3d)
project_tools.print_comparison_stats(vdf,vdf_herm_3d)

### Gaussian Mixture Model (GMM)

In [ ]:
"""
 Loads the original 3D VDF and performs Gaussian Mixture Model (GMM) decomposition.
 Input: vdf - NumPy array representing the original 3D VDF
 Output: vdf_rec Reconstructed VDF using GMM
"""
#### load original 3d vdf
vdf_3d=vdf.copy()

### define number of populations and normalization parameter
n_pop=15
norm_range=300

### RUN GMM
means,weights,covs,norm_unit=project_tools.run_gmm(vdf_3d,n_pop,norm_range)
### reconstruction resolution and limits of v_space axes
n_bins=nx
v_min,v_max=0,nx

### reconstruction of the vdf 
vdf_rec=project_tools.reconstruct_vdf(n_pop,means,covs,weights,n_bins,v_min,v_max)
vdf_rec=vdf_rec*norm_unit*norm_range
total_size =5*8+8*np.prod(np.shape(np.array(covs)))+8*np.prod(np.shape(np.array(weights)))+8*np.prod(np.shape(np.array(means)))
print(f"Compresion achieved using GMM = {round(vdf_size/total_size,2)}")

project_tools.plot_vdfs(vdf,vdf_rec)
project_tools.print_comparison_stats(vdf,vdf_rec)

## Discrete Cosine Transform (DCT)

DCT is simple to implement and is based on linear operations. This is widely used as the JPEG compression standard for images, but it can be extended to 3D distributions as well. Blockyness is apparent, and there is little room for fine-tuning: block size and number of retained DCT components.

In [ ]:
from scipy.fft import dctn, idctn

vdf_3d = vdf.copy()
orig_shape = vdf_3d.shape
vdf_3d[np.isnan(vdf_3d)] = 0

blocksize = 8
paddings = (np.ceil(np.array(vdf_3d.shape)/8)).astype(int)*8 - vdf_3d.shape
paddings = ((0,paddings[0]),(0,paddings[1]),(0,paddings[2]))
vdf_3d = np.pad(vdf_3d, paddings)

# dct_data = dctn(vdf_3d)
# print(dct_data.shape)
# vdf_rec = idctn(dct_data)
# print(vdf_rec.shape)

block_data = np.zeros_like(vdf_3d)
for i in range(0,vdf_3d.shape[0], blocksize):
    for j in range(0, vdf_3d.shape[1], blocksize):
        for k in range(0, vdf_3d.shape[2], blocksize):
            block_data[i:i+blocksize,j:j+blocksize, k:k+blocksize] = dctn(vdf_3d[i:i+blocksize,j:j+blocksize, k:k+blocksize])

keep_n = 4
zeroed = np.zeros_like(block_data)
for i in range(keep_n):
    for j in range(keep_n):
        for k in range(keep_n):
            zeroed[i::blocksize,j::blocksize,k::blocksize] = block_data[i::blocksize,j::blocksize,k::blocksize]

fig, ax = plt.subplots()
ax.pcolor(np.log(np.abs(zeroed[:,0,:])))


volume_compressed = np.prod(keep_n*np.ceil(np.array(vdf_3d.shape)/8))
volume_orig = np.prod(vdf_3d.shape)
compression = volume_orig/volume_compressed

vdf_rec = np.zeros_like(vdf_3d)
for i in range(0,vdf_3d.shape[0], blocksize):
    for j in range(0, vdf_3d.shape[1], blocksize):
        for k in range(0, vdf_3d.shape[2], blocksize):
            vdf_rec[i:i+blocksize,j:j+blocksize, k:k+blocksize] = idctn(zeroed[i:i+blocksize,j:j+blocksize, k:k+blocksize])

print("compression:", compression)
# vdf_rec = idctn(dct_data)
vdf_rec = vdf_rec[0:orig_shape[0],0:orig_shape[1],0:orig_shape[2]]

project_tools.plot_vdfs(vdf,vdf_rec)
project_tools.print_comparison_stats(vdf,vdf_rec)

## Discrete Wavelet Transform (DWT)

Here we employ the PyWavelets package for prototyping compression via Discrete Wavelet Transforms [@lee_2019].

First, we test quantizing the coefficients as a sanity check.

In [ ]:
import pywt

# print(pywt.wavelist())

vdf_3d = vdf.copy()
orig_shape = vdf_3d.shape
vdf_3d[np.isnan(vdf_3d)] = 0

comp_type = np.int8
quant = np.iinfo(comp_type).max/3
# quant = np.finfo(comp_type).max/3
# quant_min = np.iinfo(comp_type).min/3

# norm = np.max([np.nanmax(vdf_3d)/quant_max, np.nanmin(vdf_3d)/quant_min])
# print(quant_max, quant_min, norm)

norm = np.nanmax(vdf_3d)/quant

vdf_3d /= norm
vdf_3d[vdf_3d<0]=0
print(np.nanmax(vdf_3d),np.nanmin(vdf_3d), norm)


coeffs3 = pywt.dwtn(vdf_3d,'bior1.3')

# print(coeffs3)
coeffs3_compress = coeffs3.copy()

for k, v in coeffs3.items():
    print(np.min(v),np.max(v), len(v))
    coeffs3_compress[k] = v.astype(comp_type)


vdf_rec = pywt.idwtn(coeffs3_compress,'bior1.3')*norm




volume_compressed = 0
for k,v in coeffs3_compress.items():
    volume_compressed += sys.getsizeof(v)
volume_orig = 0
for k,v in coeffs3.items():
    volume_orig += sys.getsizeof(v)
compression = volume_orig/volume_compressed

print("compression:", compression)

project_tools.plot_vdfs(vdf,vdf_rec)
project_tools.print_comparison_stats(vdf,vdf_rec)

Next, we try a naive thresholding operation on the coefficients for reconstruction. The results are encouraging, and a more elaborate thresholding scheme could help with capturing details at the fringes.

In [ ]:
import pywt

print(pywt.wavelist(kind='discrete'))

vdf_3d = vdf.copy()
orig_shape = vdf_3d.shape
vdf_3d[np.isnan(vdf_3d)] = 0

comp_type = np.float32
# quant = np.iinfo(comp_type).max/3


norm = 1# np.nanmax(vdf_3d)/quant

vdf_3d /= norm
vdf_3d[vdf_3d<0]=0
print(np.nanmax(vdf_3d),np.nanmin(vdf_3d), norm)

wavelet = 'db4'#'bior1.3'

dwtn_mlevel = pywt.dwtn_max_level(vdf_3d.shape,wavelet)
level_delta = 2
print("Decomposing to ", dwtn_mlevel-level_delta, "levels out of ", dwtn_mlevel)
coeffs3 = pywt.wavedecn(vdf_3d,wavelet=wavelet, level = dwtn_mlevel-2)

# print(coeffs3)
coeffs3_comp = coeffs3.copy()
print(type(coeffs3_comp))

zeros = 0
nonzeros = 0
threshold = 1e-16
for i,a in enumerate(coeffs3_comp):
    print(type(a))
    zero_app = False
    # print(a.shape)
    if(type(a) == type(np.ndarray(1))):
        coeffs3_comp[i] = a
        mask = np.abs(a) < threshold
        zeros += np.sum(mask)
        nonzeros += np.sum(~mask)
        # nonzeros += np.prod(a.shape)
        coeffs3_comp[i][mask] = 0
    else:
        for k,v in a.items():
            mask = np.abs(v) < threshold
            coeffs3_comp[i][k] = v
            coeffs3_comp[i][k][mask] = 0
            zeros += np.sum(mask)
            nonzeros += np.sum(~mask)

print("number of zeros:", zeros, "nonzeros:", nonzeros)
vdf_rec = pywt.waverecn(coeffs3_comp,wavelet=wavelet)*norm

compression = (np.prod(vdf_3d.shape)/nonzeros)


# volume_compressed = 0
# for k,v in coeffs3_compress.items():
#     volume_compressed += sys.getsizeof(v)
# volume_orig = 0
# for k,v in coeffs3.items():
#     volume_orig += sys.getsizeof(v)
# compression = volume_orig/volume_compressed

print("compression:", compression)

project_tools.plot_vdfs(vdf,vdf_rec)
project_tools.print_comparison_stats(vdf,vdf_rec)

Lastly, stationary wavelet transforms might have desirable qualities. However, computing these is very slow with the current package, and this output is disabled for now.

In [ ]:
if False:
    import pywt

    print(pywt.wavelist(kind='discrete'))

    vdf_3d = vdf.copy()
    orig_shape = vdf_3d.shape
    vdf_3d[np.isnan(vdf_3d)] = 0

    mlevel = 6
    paddings = (np.ceil(np.array(vdf_3d.shape)/2**mlevel)).astype(int)*2**mlevel - vdf_3d.shape
    paddings = ((0,paddings[0]),(0,paddings[1]),(0,paddings[2]))
    vdf_3d = np.pad(vdf_3d, paddings)

    comp_type = np.float32
    # quant = np.iinfo(comp_type).max/3


    norm = 1# np.nanmax(vdf_3d)/quant

    vdf_3d /= norm
    vdf_3d[vdf_3d<0]=0
    # print(np.nanmax(vdf_3d),np.nanmin(vdf_3d), norm)

    wavelet = 'db4'#'bior1.3'
    coeffs3 = pywt.swtn(vdf_3d, wavelet, mlevel)

    # print(coeffs3)
    coeffs3_comp = coeffs3.copy()

    zeros = 0
    nonzeros = 0
    threshold = 1e-15
    for i,a in enumerate(coeffs3_comp):
        # print(type(a))
        # print(a.shape)
        if(type(a) == type(np.ndarray(1))):
            coeffs3_comp[i] = a
            mask = np.abs(a) < threshold
            zeros += np.sum(mask)
            nonzeros += np.sum(~mask)
            coeffs3_comp[i][mask] = 0
        else:
            for k,v in a.items():
                mask = np.abs(v) < threshold
                coeffs3_comp[i][k] = v
                coeffs3_comp[i][k][mask] = 0
                mask = np.abs(v) < threshold
                zeros += np.sum(mask)
                nonzeros += np.sum(~mask)

    print("number of zeros:", zeros, "nonzeros:", nonzeros)
    vdf_rec = pywt.iswtn(coeffs3_comp, wavelet)*norm

    compression = ((zeros+nonzeros)/nonzeros)


    # volume_compressed = 0
    # for k,v in coeffs3_compress.items():
    #     volume_compressed += sys.getsizeof(v)
    # volume_orig = 0
    # for k,v in coeffs3.items():
    #     volume_orig += sys.getsizeof(v)
    # compression = volume_orig/volume_compressed

    print("compression:", compression)
    vdf_rec = vdf_rec[0:orig_shape[0],0:orig_shape[1],0:orig_shape[2]]

    project_tools.plot_vdfs(vdf,vdf_rec)
    project_tools.print_comparison_stats(vdf,vdf_rec)